# Generate speckle

In this notebook, we will study of to generate speckle

In [ ]:
#awojdyla@lbl.gov, July 2020

In [1]:
import numpy as np
# defining simulations parameters

In [49]:
def ft(t):
    return np.fft.fftshift( np.fft.fft2(np.fft.ifftshift(t)))

In [50]:
def ift(t):
    return np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(t)))

In [51]:
def fs(t):
    return (np.arange(0,1/(t[1]-t[0]),1/((t[1]-t[0])*len(t)))) - (1/(t[1]-t[0])-np.mod(len(t),2)*1/((t[1]-t[0])*len(t)))/2

In [52]:
def propTF(E_in,L_m,lambda_m,z_m):
    #get input field array size
    (Nx, Ny)=np.shape(E_in); 
    dx=L_m/Nx; #sample interval

    #(dx<lambda.*z/L)


    fx = fs(np.arange(Nx)*dx);
    if Ny>2:
        fy = fs(np.arange(Ny)*dx);
    else:
        fy = 0;

    [FX,FY]= np.meshgrid(fx,fy);

    H=np.exp(-1j*np.pi*lambda_m*z_m*(FX**2+FY**2))

    E_out = ft(ft(E_in)*H);
    
    return E_out

In [53]:
def gaussian(x_px, mean_px, fwhm_px):
            sigma_x = fwhm_px/(2*np.sqrt(2*np.log(2)));
            return np.exp(-((x_px-mean_px)/(np.sqrt(2)*sigma_x))**2);

In [54]:
# screen size
L_m = 1e-3;
# pixel size
dx_m= 1e-6;

# define spatial axis
x_m = np.linspace(-L_m/2,L_m/2,int(np.floor(L_m/dx_m)))
N = len(x_m);

# wavelength
lambda_m = 1e-6;

# ccd position
z_m = 0.300e-3;

(X_m,Y_m) = np.meshgrid(x_m,x_m);
# define frequency axis
f_cpm = fs(x_m);
(Fx, Fy) = np.meshgrid(f_cpm,f_cpm);

In [79]:
# frequency filter
fc_cpm = 1e6;
GFILT = gaussian(Fx, 0, fc_cpm)*gaussian(Fy, 0, fc_cpm);

In [72]:
# generation of speckle and propagations
noise = np.random.randn(N,N)
noise_filt = np.abs(ift(ft(noise)*GFILT));
noise_filt = noise_filt/np.std(noise_filt);

In [73]:
# generate laser beam
E0 = gaussian(X_m, 0, 500e-6)*gaussian(Y_m, 0, 500e-6);
# add speckle to beam
E1 = E0*np.exp(1j*2*np.pi*noise_filt/20);


In [74]:
# propagation
E2 = propTF(E1,L_m,lambda_m,z_m);
I = np.abs(E2)**2